# IMPORTS

## Libraries

In [2]:
import warnings

import numpy  as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats as ss

from IPython.display      import Image
from IPython.core.display import HTML


warnings.filterwarnings("ignore")

## Load Dataset

In [3]:
dfRaw = pd.read_feather('00-Data/FeatherData/df04.feather')

In [4]:
dfRaw.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Origin
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,True,True,101348.88,True,Latin
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,False,True,112542.58,False,Latin
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,True,False,113931.57,True,Latin
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,False,False,93826.63,False,Latin
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,True,True,79084.10,False,Latin


## Helper Functions

#### Jupyter Settings

In [5]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML('<style>.container { width:100% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)
    
    sns.set()

In [6]:
jupyter_settings()  

Populating the interactive namespace from numpy and matplotlib


#### Functions

# DATA PREPARATION

In [7]:
df05 = dfRaw.copy()

## ~~Normalization~~

## Rescaling

In [9]:
# Numerical Attributes and Boolean
numAttributesAndBool = df05.select_dtypes(include=['int64', 'float64', 'bool'])
NotNumerial = ['RowNumber', 'CustomerId']
numAttributesAndBool = numAttributesAndBool[numAttributesAndBool.columns[~numAttributesAndBool.columns.isin(NotNumerial)]]
numAttributesAndBool.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,42,2,0.00,1,True,True,101348.88,True
1,608,41,1,83807.86,1,False,True,112542.58,False
2,502,42,8,159660.80,3,True,False,113931.57,True
3,699,39,1,0.00,2,False,False,93826.63,False
4,850,43,2,125510.82,1,True,True,79084.10,False


# Convert to .feather

In [26]:
df03.to_feather('00-Data/FeatherData/df01.feather')